#Step 1: Checking GPU Availability
Check if a GPU is available in your Google Colab environment. A GPU can significantly accelerate the training process of deep learning models like YOLOv8.

In [16]:
!nvidia-smi

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

#Step 2: Unzipping Dataset Files
Unzip the dataset files containing images and annotations. These files are typically compressed to save space and are now being extracted for use in training the YOLOv8 model.

**NOTE:** Ensure that the images and their corresponding annotation files have the same names (e.g., image_1.png should have an associated image_1.txt file).

In [7]:
# Unzip the zip file containing the images
!unzip -q '/content/train.zip' -d '/content/images'

# Unzip the zip file containing the annotations
!unzip -q '/content/test.zip' -d '/content/annotations'

#Step 3: Installing Ultralytics
Install Ultralytics, a library that simplifies working with YOLO object detection models.

In [4]:
# Install Ultralytics library
!pip install ultralytics

# Import necessary modules from Ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.4/904.4 kB 39.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


#Step 4: Mounting Google Drive
Mount Google Drive to the Colab environment, enabling access to files stored in your Google Drive.

In [ ]:
# Import the drive module from google.colab
from google.colab import drive

# Mount Google Drive to '/content/Google_Drive'
drive.mount('/content/Google_Drive')

#Step 5: Define Root Directory and Create Subdirectories
Define the root directory for your project and create necessary subdirectories to organize your data.

In [5]:
import os

# Prompt user for the path
PATH = input("Enter the desired path: ")

ROOT_DIR = '/content/' + PATH

# Create directory for data
DATA_DIR = os.path.join(ROOT_DIR, 'data')

# Create directory for images
IMAGES_DIR = os.path.join(DATA_DIR, 'images')
IMAGES_VAL_DIR = os.path.join(IMAGES_DIR, 'val')
IMAGES_TRAIN_DIR = os.path.join(IMAGES_DIR, 'train')

# Create directory for labels
LABELS_DIR = os.path.join(DATA_DIR, 'labels')
LABELS_VAL_DIR = os.path.join(LABELS_DIR, 'val')
LABELS_TRAIN_DIR = os.path.join(LABELS_DIR, 'train')

# Create directory for testing
TESTING_DIR = os.path.join(ROOT_DIR, 'testing')

# Create the directories if the root directory doesn't exist
if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)

    os.makedirs(DATA_DIR)

    os.makedirs(IMAGES_DIR)
    os.makedirs(IMAGES_VAL_DIR)
    os.makedirs(IMAGES_TRAIN_DIR)

    os.makedirs(LABELS_DIR)
    os.makedirs(LABELS_VAL_DIR)
    os.makedirs(LABELS_TRAIN_DIR)

    os.makedirs(TESTING_DIR)

    print(f"Root directory '{ROOT_DIR}' created successfully.")
else:
    print(f"Root directory '{ROOT_DIR}' already exists.")

Enter the desired path: object detection
Root directory '/content/object detection' created successfully.


#Step 6: Organize Dataset for Training and Validation
Organize the dataset by moving images and corresponding annotations into separate directories for training and validation.

In [9]:
import shutil

# Replace placeholders with actual paths to your images and annotations
IMAGES_PATH = '/content/images/train/images'
ANNOTATIONS_PATH = '/content/images/train/labels'

# Get list of image and annotation files
image_files = os.listdir(IMAGES_PATH)
annotation_files = os.listdir(ANNOTATIONS_PATH)

# Sort the files for consistency
image_files.sort()
annotation_files.sort()

# Determine the number of files for the training set
train_count = int(len(image_files) * 0.9) # 70% of data used for training, 30% used for validation

# Move files to the training directory
for file in image_files[:train_count]:
    shutil.move(os.path.join(IMAGES_PATH, file), os.path.join(IMAGES_TRAIN_DIR, file))
for file in annotation_files[:train_count]:
    shutil.move(os.path.join(ANNOTATIONS_PATH, file), os.path.join(LABELS_TRAIN_DIR, file))

# Move files to the validation directory
for file in image_files[train_count:]:
    shutil.move(os.path.join(IMAGES_PATH, file), os.path.join(IMAGES_VAL_DIR, file))
for file in annotation_files[train_count:]:
    shutil.move(os.path.join(ANNOTATIONS_PATH, file), os.path.join(LABELS_VAL_DIR, file))

#Step 7: Generate YAML Configuration File
Create a YAML configuration file specifying the paths to the training and validation datasets, as well as the class names (labels) used in your dataset.

In [12]:
import yaml

# Define the data structure for the YAML file
data = {
    'path': f'{DATA_DIR}',
    'train': 'images/train',
    'val': 'images/val',
   'names': {0: 'Andrew tate', 1: 'female', 2: 'long hair', 3: 'male', 4: 'short hair', 5: 'cats', 6:'dogs'
    }
}

# Construct the full path to the YAML file
output_file = os.path.join(ROOT_DIR, "config.yaml")

# Write the data to the YAML file
with open(output_file, 'w') as yaml_file:
    yaml.dump(data, yaml_file)

#Step 8: Train the YOLOv8 Model
Train the YOLOv8 model using the provided dataset configuration and save the training results.

In [13]:
# Load a pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")

# Train the model using the provided dataset configuration
model_results = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=20)

# Save the training results
shutil.make_archive(base_dir='/content/runs', root_dir='/content/runs', format='zip', base_name=f'{ROOT_DIR}/runs')

Ultralytics 8.3.56 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/object detection/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /content/object detection/data/labels/train.cache... 461 images, 3 backgrounds, 0 corrupt: 100%|██████████| 461/461 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/object detection/data/labels/val.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         52        148      0.454      0.548      0.517      0.307



20 epochs completed in 0.076 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.84it/s]


                   all         52        148      0.456      0.547      0.518      0.307
                female         52         73      0.908      0.945      0.943      0.698
             long hair         40         72       0.46      0.697       0.59       0.22
            short hair          3          3          0          0       0.02     0.0034
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train2


'/content/object detection/runs.zip'

#Step 9: View Training Results
Display the training results and the confusion matrix generated during the training process.

In [14]:
from IPython.display import Image

# Display the training results
Image('runs/detect/train2/results.png')

FileNotFoundError: No such file or directory: 'runs/detect/train/results.png'

FileNotFoundError: No such file or directory: 'runs/detect/train/results.png'

<IPython.core.display.Image object>

In [ ]:
# Display the confusion matrix
Image('runs/detect/train/confusion_matrix.png')

#Step 10: Process Video Files
Process video files using the trained YOLOv8 model and save the processed videos with bounding boxes and class labels.

**NOTE:** Ensure you have uploaded your video files to the testing folder before proceeding with this step.

In [15]:
import cv2

# Define the directory where you want to save the images
OUTPUT_DIR = '/content/output'

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Path to the trained model file
model_file = "/content/runs/detect/train/weights/best.pt"

# Load the trained model
model = YOLO(model_file)

# List all files in the folder
video_files = os.listdir(TESTING_DIR)

# Process each video file
for video_file in video_files:
    cap = cv2.VideoCapture(os.path.join(TESTING_DIR, video_file))

    # Get the dimensions and frame rate of the video
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Create the output video writer
    out = cv2.VideoWriter(f'{OUTPUT_DIR}/{video_file}_object_detection.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    ret, frame = cap.read()

    while ret:
        results = model(frame)[0]

        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result

            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

            # Display class name and score
            label = f'{results.names[int(class_id)]}: {score:.2f}'
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

        out.write(frame)
        ret, frame = cap.read()

    cap.release()
    out.release()
    cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train/weights/best.pt'

#Step 11: Display Processed Video
Use the MoviePy library to display the processed video within the Colab environment.

In [ ]:
from moviepy.editor import *

# Path to the processed video file
path = 'REPLACE_WITH_PATH_TO_VIDEO_FILE'

# Load the processed video clip
clip = VideoFileClip(path)

# Display the video clip
clip.ipython_display(height=540, width=960)